In [1]:
from sklearn.metrics import classification_report
from tensorflow.python.keras import regularizers, Model, Input, optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.layer_utils import print_summary
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import random
random.seed(32)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True
)

validation_datagen = ImageDataGenerator(
        rescale=1./255,
)
train_generator = train_datagen.flow_from_directory(
        'datasets/DFDC/train',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=True,
        seed=32) 
validation_generator = validation_datagen.flow_from_directory(
        'datasets/DFDC/val',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=True,
        seed=32) 

In [3]:
x=train_generator.__getitem__(0)
print(len(train_generator.labels),train_generator.labels)
print(x[1].shape)
print(x[1])
plt.imshow(x[0][7])
plt.title(x[1][7])


In [4]:
def load_ShallowNetV3(weights, name, model_input):
    model = ShallowNetV3(model_input)
    model.load_weights(weights)
    # model.name = name
    return model

#ShallowNet V3
def ShallowNetV3(model_input):
    # block 1
    x = Conv2D(32, (5, 5), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.0001))(model_input)
    x = Dropout(0.25)(x)
    x = Conv2D(32, (1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(32, (1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)
    # block 2
    x = Conv2D(64, (3, 3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, (1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, (1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)
    # block 3
    x = Conv2D(128, (3, 3), activation='relu', padding='valid', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(32, (1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dropout(0.25)(x)
    # block 4
    x = Flatten()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(2, activation='sigmoid')(x)
    model = Model(model_input, x, name='ShallowNetV3')
    return model

In [5]:
inputs = Input(shape=(128, 128, 3), name='image_input')
model= ShallowNetV3(inputs)

model_checkpoint_callback = ModelCheckpoint(
    filepath="SNV3_DFDC_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
csv_logger = CSVLogger("SNV3_DFDC_train.csv", append=True, separator=',')
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=115, positions=None, print_fn=None)

In [6]:
model.fit_generator(train_generator,
                    verbose=2,
                    epochs=50,
                    validation_data=validation_generator,
                    callbacks=[model_checkpoint_callback,csv_logger])
model.save_weights("SNV3_DFDC_E50.h5")

# DeepFakeDetection Test 

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)
inputs = Input(shape=(128, 128, 3), name='image_input')
model=load_ShallowNetV3("SNV3_DFDC_Ebest.h5","ShallowNetV3",inputs)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakeDetection/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
# model.load_weights("SNV3_DFDC_Ebest.h5")
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFake Test 

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
inputs = Input(shape=(128, 128, 3), name='image_input')
model=load_ShallowNetV3("SNV3_DFDC_Ebest.h5","ShallowNetV3",inputs)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFake/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
# model.load_weights("SNV3_DFDC_Ebest.h5")
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# FaceSwap Test 

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)
inputs = Input(shape=(128, 128, 3), name='image_input')
model=load_ShallowNetV3("SNV3_DFDC_Ebest.h5","ShallowNetV3",inputs)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/FaceSwap/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
# model.load_weights("SNV3_DFDC_Ebest.h5")
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# Face2Face Test 

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
inputs = Input(shape=(128, 128, 3), name='image_input')
model=load_ShallowNetV3("SNV3_DFDC_Ebest.h5","ShallowNetV3",inputs)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/Face2Face/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
# model.load_weights("SNV3_DFDC_Ebest.h5")
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# NeuralTextures Test 

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
inputs = Input(shape=(128, 128, 3), name='image_input')
model=load_ShallowNetV3("SNV3_DFDC_Ebest.h5","ShallowNetV3",inputs)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/NeuralTextures/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
# model.load_weights("SNV3_DFDC_Ebest.h5")
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFakewild Test 

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)
inputs = Input(shape=(128, 128, 3), name='image_input')
model=load_ShallowNetV3("SNV3_DFDC_Ebest.h5","ShallowNetV3",inputs)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakewild/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
# model.load_weights("SNV3_DFDC_Ebest.h5")
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DFDC Test 

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)
inputs = Input(shape=(128, 128, 3), name='image_input')
model=load_ShallowNetV3("SNV3_DFDC_Ebest.h5","ShallowNetV3",inputs)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=115, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DFDC/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
# model.load_weights("SNV3_DFDC_Ebest.h5")
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)